## Data scarping - log in task

In [0]:
# make some preparation for packages
# package use
import requests
import urllib
import urllib.request
import time
import re
from bs4 import BeautifulSoup
from IPython.core.display import HTML

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import copy
import datetime 
from collections import Counter

In [0]:
!pip install mechanize

     |████████████████████████████████| 112kB 5.0MB/s 


In [0]:
!pip install selenium

     |████████████████████████████████| 911kB 4.9MB/s 


In [0]:
!pip install scrapy

     |████████████████████████████████| 235kB 4.3MB/s 
     |████████████████████████████████| 3.1MB 31.8MB/s 
     |████████████████████████████████| 61kB 20.1MB/s 
     |████████████████████████████████| 174kB 34.9MB/s 
     |████████████████████████████████| 61kB 18.7MB/s 
     |████████████████████████████████| 2.3MB 34.5MB/s 
     |████████████████████████████████| 102kB 27.3MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=e54fb86ea8d8b0af49b83e0398678bd99efddac9699739785d6e95601ccefa38
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built PyDispatcher


In [0]:
import os
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import scrapy

## Task 1:

(1) "Film-masterfile (from 'film-merge-8th.dta').csv"
Here you can add the following variables.
 - 'releasedate': release date of each film
 - 'closedate': closing date of each film
 - 'numberofcritics': number of critics rating each film  

You can use 'bomlink' to scrape variables 'releasedate' and 'closedate'.
Given columns are already created in the csv file at the right side.

In [0]:
# read the csv file first
film_merge = pd.read_csv("Film-masterfile (from 'film-merge-8th.dta').csv")
print(film_merge.columns)

# analyze bomlink column
film_merge['bomlink'].head(5)

Index(['year', 'bomrank', 'bomtitle', 'bomlink', 'domesticgross',
       'foreigngross', 'runtimemin', 'productionbudget', 'majordistributor',
       'grated', 'rrated', 'unrated', 'remove', 'imdblink', 'studio',
       'totalusgross', 'totaltheater', 'openingusgross', 'openingtheaters',
       'foreigndm', 'languagecount', '_merge2', 'qualityfilm', 'countaward',
       '_merge3', 'genrespanning', '_merge4', 'criticratings_avg',
       'criticrating_var', 'user_avg', 'user_var', '_merge5', 'imdbfilmtitle',
       'avg_minority_gender', 'avg_minority_race', 'avg_minority',
       'avg_minority_1', 'multidirector', 'releasedate', 'closedate',
       'numberofcritics'],
      dtype='object')


0    http://www.boxofficemojo.com/movies/?id=wizard...
1    http://www.boxofficemojo.com/movies/?id=singin...
2    http://www.boxofficemojo.com/movies/?id=raider...
3    http://www.boxofficemojo.com/movies/?id=ghostb...
4    http://www.boxofficemojo.com/movies/?id=backto...
Name: bomlink, dtype: object

### Task 1.1  Fill in the release date first

In [0]:
# define a function to extract release date
def extractRelease(bomlink_, keyword_ = "date"):
  # set the url to the website and access the site
  response = requests.get(bomlink_)
  # parse the html with BeautifulSoup for data structure
  soup = BeautifulSoup(response.text, "html.parser")

  # search for href link within central table
  filter_ = soup.find("center").findAll("a")
  for record_ in filter_:
    if keyword_ in record_["href"]:
      # find release date record and cut
      return record_.text
  # records not found
  return np.nan

In [0]:
# fill in release date 
# and turn time string into timestamp
for i in range(0, film_merge.shape[0]):
  film_merge['releasedate'][i] = pd.to_datetime(extractRelease(film_merge['bomlink'][i]), 
                                                infer_datetime_format=True)
  # track process
  if i%500 == 0:
    print(i)

0
500
1000
1500
2000


In [0]:
film_merge['releasedate'].head(10)

0    2013-09-20 00:00:00
1    2012-07-12 00:00:00
2    2012-09-07 00:00:00
3    2014-08-29 00:00:00
4    2015-10-21 00:00:00
5    2013-02-08 00:00:00
6    2012-01-13 00:00:00
7    2013-04-05 00:00:00
8    2006-10-20 00:00:00
9    2011-09-16 00:00:00
Name: releasedate, dtype: object

### Task 1.2 Fill in the close date 

In [0]:
# there are three tables in general
# Total Lifetime Grosses - Domestic Summary - Charts
def extractClose(bomlink_, keyword_ = "Close Date:"):
  response = requests.get(bomlink_)
  # parse the html with BeautifulSoup for data structure
  soup = BeautifulSoup(response.text, "html.parser")

  # filter div box content to use 
  filter_ = soup.findAll("div", attrs={"class": "mp_box_content"})
  # assuming each page owns at least two tables 
  for i in range(0, len(filter_[1].findAll("td"))):
      # check available tables within 2nd box content
      if filter_[1].findAll("td")[i].get_text().replace('\xa0',' ') == keyword_:
        # Close date exists then return next string 
        date = filter_[1].findAll("td")[i+1].get_text().replace('\xa0',' ')
        return pd.to_datetime(date, infer_datetime_format=True)
  # Close date not exist
  return np.nan

In [0]:
# fill in Close date 
for i in range(0, film_merge.shape[0]):
  film_merge['closedate'][i] = extractClose(film_merge['bomlink'][i])
  # track process
  if i%500 == 0:
    print(i)

0
500
1000
1500
2000


In [0]:
film_merge['closedate'].head(10)

0    2013-10-17 00:00:00
1    2012-08-26 00:00:00
2    2012-10-04 00:00:00
3    2014-09-18 00:00:00
4    2015-10-21 00:00:00
5    2013-02-21 00:00:00
6    2012-05-03 00:00:00
7    2013-05-23 00:00:00
8    2007-01-04 00:00:00
9    2012-01-12 00:00:00
Name: closedate, dtype: object

In [0]:
# check the number of non-time records in release date and close date
print(len(film_merge[film_merge['releasedate'].isnull()]))
print("Number of films that lack close date:", len(film_merge[film_merge['closedate'].isnull()]))
film_merge[film_merge['closedate'].isnull()]['bomlink'].head(5)

0
Number of films that lack close date: 43


51     http://www.boxofficemojo.com/movies/?id=bobby.htm
58     http://www.boxofficemojo.com/movies/?id=bluebu...
114    http://www.boxofficemojo.com/movies/?id=santav...
145    http://www.boxofficemojo.com/movies/?id=mainho...
268    http://www.boxofficemojo.com/movies/?id=choris...
Name: bomlink, dtype: object

### Task 1.3 Fill number of critics for each rating film

In [0]:
# count number of critics for each rating film
def extractNumOfCritics(imdblink_):
  url = imdblink_ + 'criticreviews'
  response = requests.get(url)
  # analyze rating website information
  soup = BeautifulSoup(response.text, "html.parser")
  # No score filter - simple count
  filter_score = soup.findAll('div', {'class': ["critscore critscore_favorable", 
                           "critscore critscore_mixed", 
                           "critscore critscore_unfavorable"]})
  return len(filter_score)

In [0]:
for i in range(0, film_merge.shape[0]):
  film_merge['numberofcritics'][i] = extractNumOfCritics(film_merge['imdblink'][i])
  if i%500 == 0:
    print(i)
film_merge['numberofcritics'].describe()

0
500
1000
1500
2000


count    2376.000000
mean        9.569865
std         1.926043
min         0.000000
25%        10.000000
50%        10.000000
75%        10.000000
max        10.000000
Name: numberofcritics, dtype: float64

In [0]:
print("Number of films that lack critic reviews:", len(film_merge[film_merge['numberofcritics'].isnull()]))

Number of films that lack critic reviews: 0


In [0]:
# write document
film_merge.to_csv("Film-masterfile (from 'film-merge-8th.dta')-filled.csv")

## Task 2:
(2) "Film-cast-top5.csv"
Here you can add the following variable:
- 'starpower-actor': StarMeter power of the top5 casts, measured 10 weeks prior to the release date of each film. 

You can use the variable 'releasedate' scraped from file (1).
You can use 'actorimdb' to scrape variable 'starpower-actor'
The given column is already created in the csv file at the right side.

In [0]:
film_cast_top5 = pd.read_csv("Film-cast-top5.csv")
print(film_cast_top5.columns)
film_cast_top5.head(3)

Index(['bomfilmtitle', 'filmimdb', 'year', 'notsample', 'filmrank', 'castrank',
       'filmrole', 'actorname', 'actorimdb', 'birthyear', 'gender', 'race',
       'bomlink', 'starpower-actor'],
      dtype='object')


,bomfilmtitle,filmimdb,year,notsample,filmrank,castrank,filmrole,actorname,actorimdb,birthyear,gender,race,bomlink,starpower-actor
0,13 Going on 30,http://www.imdb.com/title/tt0337563/,2004,0,4,1,Cast,Jennifer Garner,http://www.imdb.com/name/nm0004950/,1972.0,f,white,http://www.boxofficemojo.com/movies/?id=13goin...,NaN
1,13 Going on 30,http://www.imdb.com/title/tt0337563/,2004,0,5,2,Cast,Mark Ruffalo,http://www.imdb.com/name/nm0749263/,1967.0,m,white,http://www.boxofficemojo.com/movies/?id=13goin...,NaN
2,13 Going on 30,http://www.imdb.com/title/tt0337563/,2004,0,6,3,Cast,Judy Greer,http://www.imdb.com/name/nm0339460/,1975.0,f,white,http://www.boxofficemojo.com/movies/?id=13goin...,NaN
